# ADVANCED QUERIES AND AGGREGATION

In [19]:
import pandas as pd
from datetime import datetime, timedelta
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']


In [20]:
print("Collections in eduhub:")
print(db.list_collection_names())

Collections in eduhub:
['users', 'assignments', 'enrollments', 'courses', 'submissions', 'lessons']


## COMPLEX QUERIES

### Courses with Price between $50 & $200

In [78]:
# Finding Courses with price btw $50 & &200
try:
    # Query courses in the price range
    courses_in_price_range = db.courses.find(
        {"price": {"$gte": 50, "$lte": 200}},
        {"_id": 0, "courseId": 1, "title": 1, "price": 1}
    )

    courses_list = list(courses_in_price_range)

    # Convert list to DataFrame
    df_courses_price = pd.DataFrame(courses_list)

    if not df_courses_price.empty:
        df_courses_price = df_courses_price[['courseId', 'title', 'price']]
        df_courses_price = df_courses_price.sort_values(by='price', ascending=False).reset_index(drop=True)
        
        print("Courses priced between $50 and $200:\n")
        print(df_courses_price)
    else:
        print("No courses found in the specified price range.")

except Exception as e:
    print(f"An error occurred while fetching courses by price: {e}")

Courses priced between $50 and $200:

  courseId                             title   price
0   CRS007    Machine Learning for Beginners  193.17
1   CRS008  Backend Development with Node.js  172.40
2   CRS002          Cloud Computing with AWS  162.52
3   CRS003        Full-Stack Web Development  161.31
4   CRS001   Frontend Development with React  144.35
5   CRS004  Serverless Applications on Cloud  125.72
6   CRS005      Introduction to Data Science  108.92
7   CRS012  Data Visualization with Power BI   99.99
8   CRS006   Frontend Development with React   77.36


### Users Who Joined In the Last 6 Month

In [79]:
# Finding users who join in thelast 6 month 
try:
    six_months_ago = datetime.now() - timedelta(days=6*30)  # approximate 6 months
    recent_users = db.users.find(
        {"dateJoined": {"$gte": six_months_ago}},
        {"_id": 0, "userId": 1, "firstName": 1, "lastName": 1, "dateJoined": 1}
    )
    
    recent_users_list = list(recent_users)

    # Convert to DataFrame
    df_recent_users = pd.DataFrame(recent_users_list)

    if not df_recent_users.empty:
        df_recent_users = df_recent_users[['userId', 'firstName', 'lastName', 'dateJoined']]
        df_recent_users = df_recent_users.sort_values(by='dateJoined', ascending=False).reset_index(drop=True)
        
        print("Users who joined in the last 6 months:\n")
        print(df_recent_users)
    else:
        print("No users joined in the last 6 months.")

except Exception as e:
    print(f"An error occurred while fetching recent users: {e}")

Users who joined in the last 6 months:

   userId firstName    lastName              dateJoined
0  STU020       Ada         Eze 2025-10-03 10:27:49.076
1  STU012    Daniel     Gilbert 2025-10-03 06:15:15.318
2  STU004    Martin       White 2025-09-24 19:38:59.914
3  STU010   Richard       Klein 2025-09-01 12:20:50.293
4  STU009     Harry       Smith 2025-08-12 16:56:14.552
5  STU011      Leah       Clark 2025-07-09 23:51:32.714
6  STU005     Jacob     Johnson 2025-07-01 18:37:34.293
7  STU015     David   Rodriguez 2025-06-21 17:07:15.669
8  STU014   Shannon  Villanueva 2025-04-30 08:24:14.148
9  STU008    Connie     Morales 2025-04-24 18:05:30.079


### Courses With Specific Tags

In [80]:
# Finding Courses that Have Specific Tags using $in Operator
try:
    tags_to_find = ["Python", "Data Science", "AI"]
    
    # Fetch courses with the specified tags
    courses_with_tags = db.courses.find(
        {"tags": {"$in": tags_to_find}},
        {"_id": 0, "courseId": 1, "title": 1, "tags": 1}
    )

    courses_list = list(courses_with_tags)

    # Convert list to DataFrame
    df_courses_tags = pd.DataFrame(courses_list)

    if not df_courses_tags.empty:
        # Reorder columns
        df_courses_tags = df_courses_tags[['courseId', 'title', 'tags']]
        
        print("Courses with specific tags:\n")
        print(df_courses_tags)
    else:
        print("No courses found with the specified tags.")

except Exception as e:
    print(f"An error occurred while fetching courses with tags: {e}")

Courses with specific tags:

  courseId                         title  \
0   CRS005  Introduction to Data Science   

                                                tags  
0  [machinelearning, ai, statistics, AI, Data Sci...  


### Assignments Due Next Week

In [81]:
# Finding Assignmenst due next week
try:
    one_week_from_now = datetime.now() + timedelta(days=7)
    
    upcoming_assignments = db.assignments.find(
        {"dueDate": {"$gte": datetime.now(), "$lte": one_week_from_now}},
        {"_id": 0, "assignmentId": 1, "title": 1, "dueDate": 1}
    )

    assignments_list = list(upcoming_assignments)

    # Convert list to DataFrame
    df_upcoming_assignments = pd.DataFrame(assignments_list)

    if not df_upcoming_assignments.empty:
        df_upcoming_assignments = df_upcoming_assignments[['assignmentId', 'title', 'dueDate']]
        df_upcoming_assignments = df_upcoming_assignments.sort_values(by='dueDate', ascending=True).reset_index(drop=True)
        
        print("Assignments due in the next week:")
        print(df_upcoming_assignments)
    else:
        print("No assignments are due in the next week.")

except Exception as e:
    print(f"An error occurred while fetching upcoming assignments: {e}")

No assignments are due in the next week.


## AGGREGATION PIPELINE

### COURSE ENROLLMENT STATISTICS

### a) Total Enrollment Per Course

In [ ]:
# Counting total enrollments per course
try:
    # Aggregate total enrollments per course with course info
    enrollments_per_course = db.enrollments.aggregate([
        {
            "$group": {
                "_id": "$courseId",
                "TotalEnrollments": {"$sum": 1}
            }
        },
        {
            "$lookup": {
                "from": "courses",
                "localField": "_id",
                "foreignField": "courseId",
                "as": "course"
            }
        },
        {"$unwind": "$course"},
        {
            "$project": {
                "_id": 0,
                "CourseId": "$_id",
                "Title": "$course.title",
                "TotalEnrollments": 1
            }
        },
        {"$sort": {"TotalEnrollments": -1}}
    ])

    # Convert to DataFrame
    df_enrollments = pd.DataFrame(list(enrollments_per_course))
    df_enrollments = df_enrollments[["CourseId", "Title", "TotalEnrollments"]]

    print("Total enrollments per course:\n")
    print(df_enrollments)

except Exception as e:
    print(f"Error fetching enrollments per course: {e}")

Total enrollments per course:

  CourseId                             Title  TotalEnrollments
0   CRS002          Cloud Computing with AWS                 3
1   CRS007    Machine Learning for Beginners                 3
2   CRS004  Serverless Applications on Cloud                 3
3   CRS006   Frontend Development with React                 2
4   CRS005      Introduction to Data Science                 1
5   CRS008  Backend Development with Node.js                 1
6   CRS003        Full-Stack Web Development                 1


### b) Average Course Rating

In [73]:
#  Calculating average course rating
try:
    # Aggregate average rating per course with course info
    avg_course_rating = db.submissions.aggregate([
        {
            "$group": {
                "_id": "$courseId",
                "AverageRating": {"$avg": "$grade"}
            }
        },
        {
            "$lookup": {
                "from": "courses",
                "localField": "_id",
                "foreignField": "courseId",
                "as": "course"
            }
        },
        {"$unwind": "$course"},
        {
            "$project": {
                "_id": 0,
                "CourseId": "$_id",
                "Title": "$course.title",
                "AverageRating": {"$round": ["$AverageRating", 2]}
            }
        },
        {"$sort": {"AverageRating": -1}}
    ])

    # Convert to DataFrame
    df_avg_ratings = pd.DataFrame(list(avg_course_rating))

    print("Average course rating per course:\n")
    print(df_avg_ratings)

except Exception as e:
    print(f"Error fetching average course rating: {e}")

Average course rating per course:

  CourseId                             Title  AverageRating
0   CRS003        Full-Stack Web Development           89.0
1   CRS002          Cloud Computing with AWS           88.0
2   CRS007    Machine Learning for Beginners           81.0
3   CRS004  Serverless Applications on Cloud           80.0
4   CRS008  Backend Development with Node.js           74.5
5   CRS005      Introduction to Data Science           64.0
6   CRS006   Frontend Development with React           53.0


### c)  Group by Course Category 

In [76]:
#  Group by course category (average rating per category)
try:
    avg_rating_by_category = db.courses.aggregate([
        {
            "$lookup": {
                "from": "submissions",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "grades"
            }
        },
        {"$unwind": "$grades"},
        {
            "$group": {
                "_id": "$category",
                "AverageRating": {"$avg": "$grades.grade"},
                "TotalCourses": {"$sum": 1}
            }
        },
        {
            "$project": {
                "_id": 0,
                "Category": "$_id",           
                "AverageRating": {"$round": ["$AverageRating", 2]},
                "TotalCourses": 1
            }
        },
        {"$sort": {"AverageRating": -1}}
    ])

    # Convert to DataFrame
    df_avg_rating_category = pd.DataFrame(list(avg_rating_by_category))
    df_avg_rating_category = df_avg_rating_category[["Category", "TotalCourses", "AverageRating"]]
    
    print("Average rating by category:\n")
    print(df_avg_rating_category)

except Exception as e:
    print(f"Error fetching rating by category: {e}")

Average rating by category:

          Category  TotalCourses  AverageRating
0  Cloud Computing             4          84.00
1  Web Development             4          72.75
2     Data Science             4          72.50


### STUDENT PERFORMANCE ANALYSIS

### a) Average Grade per Student

In [70]:
# Calculating Average grade per student
try:
    # Aggregate average grade per student with student info
    avg_grade_per_student = db.submissions.aggregate([
        {
            "$group": {
                "_id": "$studentId",
                "averageGrade": {"$avg": "$grade"}
            }
        },
        {"$sort": {"averageGrade": -1}},
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "userId",
                "as": "student"
            }
        },
        {"$unwind": "$student"},
        {
            "$project": {
                "_id": 0,
                "FirstName": "$student.firstName",
                "LastName": "$student.lastName",
                "Email": "$student.email",
                "AverageGrade": {"$round": ["$averageGrade", 2]}
            }
        }
    ])

    # Convert to DataFrame
    df_avg_grade_student = pd.DataFrame(list(avg_grade_per_student))

    print("Average grade per student:\n")
    print(df_avg_grade_student)

except Exception as e:
    print(f"Error fetching average grade per student: {e}")

Average grade per student:

  FirstName LastName                        Email  AverageGrade
0      Lisa   Turner       daviserica@example.com          88.0
1    Daniel  Gilbert      harryfoster@example.com          78.5
2    Connie  Morales          wramsey@example.net          76.5
3     Angel    Clark           smccoy@example.net          76.5
4   Michael   Daniel  stephenslatasha@example.com          53.0
5     Harry    Smith          frivera@example.org           NaN
6      Alan    Brown      tammyashley@example.net           NaN


### b) Completion Rate by Course

In [69]:
# Calculating Completion Rate by course
try:
    # Aggregate completion rate per course
    completion_rate = db.submissions.aggregate([
        {
            "$group": {
                "_id": "$courseId",
                "completed": {
                    "$sum": {"$cond": [{"$eq": ["$status", "graded"]}, 1, 0]}
                },
                "totalSubmissions": {"$sum": 1}
            }
        },
        {
            "$project": {
                "_id": 1,
                "completionRate": {
                    "$multiply": [
                        {"$divide": ["$completed", "$totalSubmissions"]},
                        100
                    ]
                }
            }
        },
        {
            "$lookup": {
                "from": "courses",
                "localField": "_id",
                "foreignField": "courseId",
                "as": "course"
            }
        },
        {"$unwind": "$course"},
        {
            "$project": {
                "_id": 0,
                "CourseId": "$_id",
                "Title": "$course.title",
                "CompletionRate": {"$round": ["$completionRate", 2]}
            }
        },
        {"$sort": {"CompletionRate": -1}}
    ])

    # Convert to DataFrame
    df_completion = pd.DataFrame(list(completion_rate))

    if not df_completion.empty:
        print("Completion rate by course:\n")
        print(df_completion)
    else:
        print("No submissions found for completion rate.")

except Exception as e:
    print(f"Error fetching completion rate: {e}")

Completion rate by course:

  CourseId                             Title  CompletionRate
0   CRS008  Backend Development with Node.js          100.00
1   CRS004  Serverless Applications on Cloud          100.00
2   CRS005      Introduction to Data Science          100.00
3   CRS006   Frontend Development with React          100.00
4   CRS003        Full-Stack Web Development          100.00
5   CRS007    Machine Learning for Beginners           33.33
6   CRS002          Cloud Computing with AWS           33.33


### c) Top-performing Students

In [67]:
# Calculating Top performing students
try:
    # Aggregate top 5 students by average grade and include student info
    top_students = db.submissions.aggregate([
        {
            "$group": {
                "_id": "$studentId",
                "averageGrade": {"$avg": "$grade"}
            }
        },
        {"$sort": {"averageGrade": -1}},
        {"$limit": 5},
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "userId",
                "as": "student"
            }
        },
        {"$unwind": "$student"},
        {
            "$project": {
                "_id": 0,
                "FirstName": "$student.firstName",
                "LastName": "$student.lastName",
                "Email": "$student.email",
                "AverageGrade": {"$round": ["$averageGrade", 2]}
            }
        }
    ])

    # Convert to DataFrame
    df_top_students = pd.DataFrame(list(top_students))

    print("Top Performing Students:\n")
    print(df_top_students)

except Exception as e:
    print(f"Error fetching top students: {e}")

Top Performing Students:

  FirstName LastName                        Email  AverageGrade
0      Lisa   Turner       daviserica@example.com          88.0
1    Daniel  Gilbert      harryfoster@example.com          78.5
2    Connie  Morales          wramsey@example.net          76.5
3     Angel    Clark           smccoy@example.net          76.5
4   Michael   Daniel  stephenslatasha@example.com          53.0


### INSTRUCTOR ANALYTICS

### a) Total Students Taught by each Instructor

In [ ]:
# Calculating Total Student taught by each instructor
try:
    students_per_instructor = db.courses.aggregate([
        {
            "$lookup": {
                "from": "enrollments",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "enrolled"
            }
        },
        {
            "$project": {
                "instructorId": 1,
                "studentCount": {"$size": "$enrolled"}  
            }
        },
        {
            "$group": {
                "_id": "$instructorId",
                "totalStudents": {"$sum": "$studentCount"}  
            }
        },
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "userId",
                "as": "instructor"
            }
        },
        {"$unwind": "$instructor"},
        {
            "$project": {
                "_id": 0,
                "FirstName": "$instructor.firstName",
                "LastName": "$instructor.lastName",
                "Email": "$instructor.email",
                "TotalStudents": "$totalStudents"
            }
        },
        {"$sort": {"TotalStudents": -1}}
    ])

    # Convert to DataFrame
    df_students_per_instructor = pd.DataFrame(list(students_per_instructor))

    print("Total students taught by each instructor:\n")
    print(df_students_per_instructor)

except Exception as e:
    print(f"Error fetching students per instructor: {e}")

Total students taught by each instructor:

  FirstName   LastName                      Email  TotalStudents
0     Terri       Reid         chad43@example.com              5
1      Sean      Ellis        asnyder@example.org              4
2     Emily  Hernandez        robin73@example.com              3
3   Matthew      Davis  garciastephen@example.net              2


### b) Average Course Rating per Instructor

In [ ]:
# Calculating Average Course rating  per Instructor
try:
    avg_rating_per_instructor = db.courses.aggregate([
        {
            "$lookup": {
                "from": "submissions",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "grades"
            }
        },
        {"$unwind": "$grades"},
        {
            "$group": {
                "_id": "$instructorId",
                "averageRating": {"$avg": "$grades.grade"}
            }
        },
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "userId",
                "as": "instructor"
            }
        },
        {"$unwind": "$instructor"},
        {
            "$project": {
                "_id": 0,
                "InstructorID": "$_id",
                "FirstName": "$instructor.firstName",
                "LastName": "$instructor.lastName",
                "Email": "$instructor.email",
                "AverageRating": {"$round": ["$averageRating", 2]}
            }
        },
        {"$sort": {"AverageRating": -1}}
    ])

    # Convert to DataFrame
    df_avg_rating = pd.DataFrame(list(avg_rating_per_instructor))

    print("Average course rating per instructor:\n")
    print(df_avg_rating)

except Exception as e:
    print(f"Error fetching average rating per instructor: {e}")

Average course rating per instructor:

  InstructorID FirstName   LastName                      Email  AverageRating
0      INST005     Emily  Hernandez        robin73@example.com          88.00
1      INST002     Terri       Reid         chad43@example.com          77.67
2      INST001      Sean      Ellis        asnyder@example.org          76.67
3      INST003   Matthew      Davis  garciastephen@example.net          53.00


### c) Revenue Generated per Instructor

In [68]:
# Total Revenue generated per Instrictor
try:
    # Aggregate total revenue per instructor
    revenue_per_instructor = db.courses.aggregate([
        {
            "$lookup": {
                "from": "enrollments",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "enrolled"
            }
        },
        {
            "$project": {
                "instructorId": 1,
                "revenue": {"$multiply": [{"$size": "$enrolled"}, "$price"]}
            }
        },
        {
            "$group": {
                "_id": "$instructorId",
                "totalRevenue": {"$sum": "$revenue"}
            }
        },
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "userId",
                "as": "instructor"
            }
        },
        {"$unwind": "$instructor"},
        {
            "$project": {
                "_id": 0,
                "FirstName": "$instructor.firstName",
                "LastName": "$instructor.lastName",
                "Email": "$instructor.email",
                "TotalRevenue": {"$round": ["$totalRevenue", 2]}
            }
        },
        {"$sort": {"TotalRevenue": -1}}
    ])

    # Convert to DataFrame
    df_revenue = pd.DataFrame(list(revenue_per_instructor))

    print("Revenue per Instructor:\n")
    print(df_revenue)

except Exception as e:
    print(f"Error fetching revenue per instructor: {e}")

Revenue per Instructor:

  FirstName   LastName                      Email  TotalRevenue
0      Sean      Ellis        asnyder@example.org        751.91
1     Terri       Reid         chad43@example.com        647.39
2     Emily  Hernandez        robin73@example.com        487.56
3   Matthew      Davis  garciastephen@example.net        154.72


### ADVANCED ANALTICS

### a) Monthly Enrollment Trends

In [ ]:
# Calculating Monthly trends
try:
    monthly_trends = db.enrollments.aggregate([
        {
            "$group": {
                "_id": {"year": {"$year": "$enrolledAt"}, "month": {"$month": "$enrolledAt"}},
                "totalEnrollments": {"$sum": 1}
            }
        },
        {"$sort": {"_id.year": 1, "_id.month": 1}}
    ])

    trends_list = list(monthly_trends)

    if trends_list:
        # Convert to DataFrame
        df_trends = pd.DataFrame(trends_list)

        # Flatten the _id dictionary to separate columns
        df_trends['Year'] = df_trends['_id'].apply(lambda x: x['year'])
        df_trends['Month'] = df_trends['_id'].apply(lambda x: x['month'])
        df_trends = df_trends.drop(columns=['_id'])

        # Reorder columns
        df_trends = df_trends[['Year', 'Month', 'totalEnrollments']]

        print("Monthly Enrollment Trends:\n")
        print(df_trends)
    else:
        print("No enrollment data found.")

except Exception as e:
    print(f"Error fetching monthly enrollment trends: {e}")

Monthly Enrollment Trends:

   Year  Month  totalEnrollments
0  2024     10                 4
1  2024     12                 2
2  2025      1                 1
3  2025      3                 1
4  2025      5                 2
5  2025      7                 1
6  2025      8                 1
7  2025      9                 2
8  2025     10                 1


### b) Most Popular Course Categories

In [61]:
# Calculating Most Popular Categories
try:
    popular_categories = db.enrollments.aggregate([
        {
            "$lookup": {
                "from": "courses",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "course"
            }
        },
        {"$unwind": "$course"},
        {
            "$group": {
                "_id": "$course.category",
                "totalEnrollments": {"$sum": 1}
            }
        },
        {"$sort": {"totalEnrollments": -1}}
    ])

    categories_list = list(popular_categories)

    if categories_list:
        # Convert to DataFrame
        df_categories = pd.DataFrame(categories_list)

        # Rename columns for clarity
        df_categories = df_categories.rename(columns={"_id": "Category", "totalEnrollments": "TotalEnrollments"})

        print("Most popular Course Categories:\n")
        print(df_categories)
    else:
        print("No enrollment data found.")

except Exception as e:
    print(f"Error fetching popular course categories: {e}")

Most popular Course Categories:

          Category  TotalEnrollments
0  Cloud Computing                 6
1     Data Science                 4
2  Web Development                 4


### c) Student Engagement Metrics

In [ ]:
# Student Engagement Metric
try:
    engagement_metrics = db.submissions.aggregate([
        {
            "$group": {
                "_id": "$studentId",
                "totalSubmissions": {"$sum": 1}
            }
        },
        {"$sort": {"totalSubmissions": -1}}
    ])

    engagement_list = list(engagement_metrics)

    if engagement_list:
        df_engagement = pd.DataFrame(engagement_list)

        # Rename columns for clarity
        df_engagement = df_engagement.rename(columns={"_id": "StudentID", "totalSubmissions": "TotalSubmissions"})

        print("Student Engagement Metrics:\n")
        print(df_engagement)
    else:
        print("No submission data found.")

except Exception as e:
    print(f"Error fetching student engagement metrics: {e}")

Student Engagement Metrics:

  StudentID  TotalSubmissions
0    STU012                 4
1    STU008                 2
2    STU013                 2
3    STU009                 1
4    STU006                 1
5    STU007                 1
6    STU001                 1
